# Advertisement Marketing: Regression

In [194]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## 1. Dataset 불러오기

In [195]:
df = pd.read_csv("VMI_data전처리.csv",index_col='Part Number')
df

,2021-09-13 18:30:00,2021-09-14 06:05:00,2021-09-14 06:25:00,2021-09-14 06:34:00,2021-09-14 17:30:00,2021-09-15 06:20:00,2021-09-15 06:37:00,2021-09-15 16:33:00,2021-09-16 06:32:00,2021-09-16 06:38:00,...,2021-10-28 15:30:00,2021-10-28 16:26:00,2021-10-29 06:30:00,2021-10-29 06:32:00,2021-10-29 07:06:00,2021-10-29 12:34:00,2021-10-29 15:03:00,2021-10-29 15:38:00,2021-10-30 07:04:00,2021-11-01 07:03:00
Part Number,,,,,,,,,,,,,,,,,,,,,
Part 0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0.0
Part 1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0.0
Part 10,16,16,16.0,16.0,16.0,16.0,16.0,16.0,23.0,23.0,...,178.0,178.0,178.0,156,156,156,156,156,156,104.0
Part 100,38,38,35.0,35.0,35.0,40.0,40.0,40.0,38.0,38.0,...,33.0,33.0,33.0,23,23,23,23,23,23,0.0
Part 101,0,0,0.0,0.0,0.0,77.0,77.0,77.0,78.0,78.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Part 95,113,113,209.0,209.0,209.0,209.0,209.0,209.0,288.0,288.0,...,276.0,276.0,276.0,264,264,264,264,264,264,209.0
Part 96,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,6.0,6.0,0,0,0,0,0,0,0.0
Part 97,127,127,32.0,32.0,32.0,28.0,28.0,28.0,28.0,28.0,...,4.0,4.0,4.0,3,3,3,3,3,3,1.0


In [196]:
df_no_zeros = df.replace(0, np.nan)

In [197]:
def log_transform(x):
    x = np.where(x <= 0, 1e-8, x)
    return np.log(x)

In [198]:
df_transformed = df.apply(log_transform)

In [199]:
df_transformed.to_csv("log_transformed_data.csv", index=True)

In [200]:
df = pd.read_csv("log_transformed_data.csv", index_col=0)
df.head()

,2021-09-13 18:30:00,2021-09-14 06:05:00,2021-09-14 06:25:00,2021-09-14 06:34:00,2021-09-14 17:30:00,2021-09-15 06:20:00,2021-09-15 06:37:00,2021-09-15 16:33:00,2021-09-16 06:32:00,2021-09-16 06:38:00,...,2021-10-28 15:30:00,2021-10-28 16:26:00,2021-10-29 06:30:00,2021-10-29 06:32:00,2021-10-29 07:06:00,2021-10-29 12:34:00,2021-10-29 15:03:00,2021-10-29 15:38:00,2021-10-30 07:04:00,2021-11-01 07:03:00
Part Number,,,,,,,,,,,,,,,,,,,,,
Part 0,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,...,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681
Part 1,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,...,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681
Part 10,2.772589,2.772589,2.772589,2.772589,2.772589,2.772589,2.772589,2.772589,3.135494,3.135494,...,5.181784,5.181784,5.181784,5.049856,5.049856,5.049856,5.049856,5.049856,5.049856,4.644391
Part 100,3.637586,3.637586,3.555348,3.555348,3.555348,3.688879,3.688879,3.688879,3.637586,3.637586,...,3.496508,3.496508,3.496508,3.135494,3.135494,3.135494,3.135494,3.135494,3.135494,-18.420681
Part 101,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,4.343805,4.343805,4.343805,4.356709,4.356709,...,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681,-18.420681


In [201]:
df.shape

(117, 153)

## 2. Train/Test Split

In [202]:
X = df.drop('2021-11-01 07:03:00', axis=1).values.astype(np.float32)
y = df['2021-11-01 07:03:00'].values.astype(np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.8,
                                                    random_state=100)

X_train.shape, X_test.shape

((93, 152), (24, 152))

In [203]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## 3. DataLoader

In [204]:
BATCH_SIZE=1
NUM_FEATURES=X.shape[1]
LEARNING_RATE=0.01
EPOCHS=50

In [205]:
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)

train_ds = TensorDataset(X_train, y_train)
train_dl = DataLoader(train_ds,
                      batch_size=BATCH_SIZE,
                      shuffle=True)

## 4. Model Build

### 4.1 nn.Sequential( )

In [206]:
model_s = nn.Sequential(nn.Linear(NUM_FEATURES, 10, bias=True), # layer 1
                      nn.ReLU(),
                      nn.Linear(10, 10, bias=True), # layer 2
                      nn.ReLU(),
                      nn.Linear(10, 10, bias=True), # layer 3
                      nn.ReLU(),
                      nn.Linear(10, 1, bias=True)) # output leayr=1 회귀

### 4.2 class

In [207]:
class Regression(nn.Module):
    def __init__(self, num_features):
        super(Regression, self).__init__()
        
        self.layer1 = nn.Linear(num_features, 10)
        self.layer2 = nn.Linear(10, 10)
        self.layer3 = nn.Linear(10, 10)
        self.layer_out = nn.Linear(10, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        
        x = self.layer2(x)
        x = self.relu(x)
        
        x = self.layer3(x)
        x = self.relu(x)
        
        x = self.layer_out(x)
        return x

In [208]:
model_c = Regression(NUM_FEATURES)
print(model_c)

Regression(
  (layer1): Linear(in_features=152, out_features=10, bias=True)
  (layer2): Linear(in_features=10, out_features=10, bias=True)
  (layer3): Linear(in_features=10, out_features=10, bias=True)
  (layer_out): Linear(in_features=10, out_features=1, bias=True)
  (relu): ReLU()
)


In [209]:
print(model_s)

Sequential(
  (0): Linear(in_features=152, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=10, bias=True)
  (3): ReLU()
  (4): Linear(in_features=10, out_features=10, bias=True)
  (5): ReLU()
  (6): Linear(in_features=10, out_features=1, bias=True)
)


## 5. Loss & Optimizer

In [210]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_s.parameters(), lr=LEARNING_RATE)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_c.to(device)

Regression(
  (layer1): Linear(in_features=152, out_features=10, bias=True)
  (layer2): Linear(in_features=10, out_features=10, bias=True)
  (layer3): Linear(in_features=10, out_features=10, bias=True)
  (layer_out): Linear(in_features=10, out_features=1, bias=True)
  (relu): ReLU()
)

## 6. Training

In [211]:
for epoch in range(EPOCHS+1):
    epoch_loss=0.0
    for X_batch, y_batch in train_dl:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        pred = model_s(X_batch)
        loss = criterion(pred, y_batch)
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
        epoch_loss += loss.item()
        
    if epoch % 10 == 0:
        print(f'Epoch {epoch}: Loss {epoch_loss/len(train_dl):.3f}')       

C:\anaconda_3\Lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0: Loss 54.946
Epoch 10: Loss 6.449
Epoch 20: Loss 3.616
Epoch 30: Loss 3.834
Epoch 40: Loss 5.585
Epoch 50: Loss 1.326


## 7. Inference & Metric

In [212]:
X_test = torch.from_numpy(X_test)
X_test = X_test.to(device)

y_preds = model_c(X_test).detach().cpu().numpy()

In [213]:
print('*MAE:', mean_absolute_error(y_test, y_preds))
print('*RMSE:', np.sqrt(mean_squared_error(y_test, y_preds)))
print('*R_Square:', r2_score(y_test, y_preds))

*MAE: 4.354197
*RMSE: 5.485193
*R_Square: -0.3561674021618839
